## 1c. Calculate <i>P<sup>v</sup></i><sub>sat</sub> for analyses in a csv file using user specified options

What if I don't want to use the default options...

## Setting things up
You need to install VolFe once on your machine, if you haven't yet. Then we need to import a few Python packages (including VolFe). 

In [1]:
# Install VolFe on your machine. Don't remove the # from this line!
# pip install VolFe # Remove the first # in this line if you have not installed VolFe on your machine before.

# import python packages
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import VolFe as vf

In that case, we create a dataframe telling VolFe what to use instead. There are lots of options that can be changed, which can be viewed below. 

In [2]:
help(vf.make_df_and_add_model_defaults)

Help on function make_df_and_add_model_defaults in module VolFe.model_dependent_variables:

make_df_and_add_model_defaults(models)
    Converts user-provided model configurations (e.g. ['carbon dioxide','MORB_Dixon95'],['hydrogen sulfide','basaltic andesite']
    into a structured pandas DataFrame, combined with default options for anything not specified
    
    
    Parameters
    ----------
    models : list of [str, str]
        A list of lists, where each inner list contains two elements: the model type (str)
        and the user-specified option (str) for that model type.
    
    Returns
    -------
    pandas.DataFrame
        A DataFrame where the first column is 'type', set as the index, and the second column
        is 'option', containing the user-specified option or the default option if none is provided.
        
    
    Model Parameters and Options
    ---------------------------------
    The following parameters can be overridden in models. 
    Any parameter can be s

Let's say I just want to use a different solubility constant for carbon dioxide and hydrogen sulfide and treat S2 as an ideal gas.

In [3]:
# choose the options I want - everything else will use the default options
my_models = [['carbon dioxide','Basalt_Dixon97'],['hydrogen sulfide','BasalticAndesite_Hughes24'],['y_S2','ideal']]

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)

# show what the model dataframe looks like
print(my_models)

                            option
type                              
COH_species     yes_H2_CO_CH4_melt
H2S_m                         True
species X                       Ar
Hspeciation                   none
fO2                       Kress91A
...                            ...
error                          0.1
print status                    no
output csv                    True
setup                        False
high precision               False

[64 rows x 1 columns]


We'll use the examples_marianas csv in files again from Brounce et al. (2014, 2016) and Kelley & Cottrell (2012).

In [4]:
# Read csv to define melt composition
my_analyses = pd.read_csv("../files/example_marianas.csv") 

# run the calculations
results1 = vf.calc_Pvsat(my_analyses,models=my_models)

KeyError: 'T_C'

For comparison, we'll also run using the default options.

In [ ]:
# run the calculations 
results2 = vf.calc_Pvsat(my_analyses)

And we can plot the comparison below: default options in red, our options in blue.

In [ ]:
fig = make_subplots(rows=1, cols=3, shared_yaxes = True, shared_xaxes = False,vertical_spacing=0.08, horizontal_spacing=0.05)

lw = 3
syms = 10
lc1 = "steelblue"
lc2 = "firebrick"

data1 = results1 # out model options
data2 = results2 # default options

r = 1
# P vs. CO2
c = 1
fig.add_trace(go.Scatter(mode = "markers", x=data1['CO2T-eq_ppmw'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "markers", x=data2['CO2T-eq_ppmw'], y=data2['P_bar'], line_color = lc2, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "CO<sub>2</sub>-eq (ppmw)", row = r, col = c)
fig.update_yaxes(title = "<i>P</i> (bars)", autorange = 'reversed', row = r, col = c)

# P vs. H2O
c = 2
fig.add_trace(go.Scatter(mode = "markers", x=data1['H2OT-eq_wtpc'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "markers", x=data2['H2OT-eq_wtpc'], y=data2['P_bar'], line_color = lc2, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "H<sub>2</sub>O-eq (wt%)", row = r, col = c)

# P vs. S
c = 3
fig.add_trace(go.Scatter(mode = "markers", x=data1['ST_ppmw'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "markers", x=data2['ST_ppmw'], y=data2['P_bar'], line_color = lc2, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "S<sub>T</sub> (ppmw)", row = r, col = c)

fig.update_layout(height=400, width=1000, plot_bgcolor='rgb(255,255,255)' , showlegend = False)

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="inside", ticklen=5, title_standoff = 0, tickcolor="black",title_font=dict(size=15, family='Helvetica', color='black'), tickfont=dict(family='Helvetica', color='black', size=12))
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="inside", ticklen=5, title_standoff = 0, tickcolor="black",title_font=dict(size=15, family='Helvetica', color='black'), tickfont=dict(family='Helvetica', color='black', size=12))

fig.update_layout(font_family="Helvetica",font_color="black")

fig.show()

Alternatively, you can load a csv with the model options listed in it (see the example models.csv in the files).

In [ ]:
my_models = pd.read_csv("../files/models.csv", index_col = [0])
print(my_models)

In [ ]:
# run the calculations
vf.calc_Pvsat(my_analyses,models=my_models)

That applied the chosen options to all samples we ran the calculation for. But what if you want to use different model dependent variables for different analyses?

### WORK IN PROGRESS ###

To do that, you specify the option you want in the same dataframe as the analyses as an extra column with the column header as the "type" of the option you're specifying. You then tell VolFe in the models dataframe to look in the analyses dataframe for the option to use by specifying "setup". 

For instance, in the following example we're going to specify the carbon dioxide solubility option in the csv file (header = carbon dioxde) and all other options are going to be default. If you look in the examples_marianas.csv file you'll see the final column is called "carbon dioxide" and different options are specified in each column. 

In [ ]:
# show the carbon dioxide options in the examples_marianas.csv
print(my_analyses[["Sample","carbon dioxide"]])

We create a new models dataframe to tell VolFe where to look for the "carbon dioxide" option and that some options are specified in the setup file:

In [ ]:
# choose the options I want - everything else will use the default options
my_models = [['carbon dioxide','setup'],['setup','yes']]

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)

# show what the model dataframe looks like
print(my_models)

In [ ]:
# run the calculations
#vf.calc_Pvsat(my_analyses,models=my_models,last_row=10)